# if event_time difference is greater than 30 it is a session find no_of event in each session ,session_id ,start_time ,end_time and duration of sessions

In [1]:
import duckdb

In [2]:
duckdb.sql("""
create or replace table session as
           select * from read_csv(User_Event_Data.csv)
""")

In [7]:
duckdb.sql("""
           with cte1 as 
                (select *,
                datediff('minutes',event_time , lag(event_time,1,event_time) over (partition by user_id order by event_time desc)) as event_diff
                    from session)
           ,cte2 as
           (select *,
           case when event_diff >30 then 1 else 0 end as session_flag,
           sum(case when event_diff >30 then 1 else 0 end ) over ( partition by user_id order by event_time) as group_id
           
           
           from cte1)

           select user_id,
           row_number()over(partition by group_id ) as session_id
           ,min(event_time) as start_time,max(event_time) as end_time,
            abs(date_diff('minutes',max(event_time),min(event_time))) as session_duration,
           count(*) as event_no
            from cte2
           group by user_id,group_id
           order by user_id,group_id
          
           

           
           """)

┌─────────┬────────────┬────────────────────────────┬────────────────────────────┬──────────────────┬──────────┐
│ user_id │ session_id │         start_time         │          end_time          │ session_duration │ event_no │
│ varchar │   int64    │         timestamp          │         timestamp          │      int64       │  int64   │
├─────────┼────────────┼────────────────────────────┼────────────────────────────┼──────────────────┼──────────┤
│ USR001  │         19 │ 2025-07-29 13:56:55.755506 │ 2025-07-29 13:56:55.755506 │                0 │        1 │
│ USR001  │          2 │ 2025-07-29 14:49:23.755506 │ 2025-07-29 14:49:23.755506 │                0 │        1 │
│ USR001  │         13 │ 2025-07-29 16:12:00.755506 │ 2025-07-29 16:12:00.755506 │                0 │        1 │
│ USR001  │         13 │ 2025-07-29 16:56:12.755506 │ 2025-07-29 16:56:12.755506 │                0 │        1 │
│ USR001  │          8 │ 2025-07-29 17:47:03.755506 │ 2025-07-29 17:47:03.755506 │              